In [25]:
import os
import numpy as np

from django.conf import settings
from collections import Counter

from koe.models import *
from koe.model_utils import get_or_error
from koe.ts_utils import bytes_to_ndarray, get_rawdata_from_binary


import plotly.plotly as py
import plotly.graph_objs as go

import plotly


def get_labels_by_sids(sids, label_level, annotator, min_occur):
    sid2lbl = {
        x: y.lower() for x, y in ExtraAttrValue.objects
        .filter(attr__name=label_level, owner_id__in=sids, user=annotator)
        .values_list('owner_id', 'value')
    }

    occurs = Counter(sid2lbl.values())

    segment_to_labels = {}
    for segid, label in sid2lbl.items():
        if occurs[label] >= min_occur:
            segment_to_labels[segid] = label

    labels = []
    no_label_ids = []
    for id in sids:
        label = segment_to_labels.get(id, None)
        if label is None:
            no_label_ids.append(id)
            labels.append('__NONE__')
        else:
            labels.append(label)

    return np.array(labels), np.array(no_label_ids, dtype=np.int32)


tensor_name = '9fc0cd0706914a62b41f67b9360d8833'
tensor = get_or_error(DerivedTensorData, dict(name=tensor_name))
database_name = tensor.database.name
annotator = tensor.annotator
label_level = 'label'

bytes_path = os.path.join(settings.BASE_DIR, tensor.get_bytes_path())
if not os.path.isfile(bytes_path):
    bytes_path = os.path.join(settings.BASE_DIR, tensor.full_tensor.get_bytes_path())
    
full_sids_path = os.path.join(settings.BASE_DIR, tensor.full_tensor.get_sids_path())
sids = bytes_to_ndarray(full_sids_path, np.int32)
coordinate = get_rawdata_from_binary(bytes_path, len(sids))

labels, no_label_ids = get_labels_by_sids(sids, label_level, annotator, 1)

unique_labels = np.unique(labels)
ndims = min(coordinate.shape[1], 3)
scatterClass = go.Scatter3d
if ndims == 2:
    scatterClass = go.Scatter


traces = []
for label in unique_labels:
    ind = np.where(labels==label)
    x = coordinate[ind, 0].ravel();
    y = coordinate[ind, 1].ravel();

    coords = dict(
        x=coordinate[ind, 0].ravel(),
        y=coordinate[ind, 1].ravel()
    )
    if ndims == 3:
        coords['z'] = coordinate[ind, 2].ravel()

    ids = sids[ind];

    trace = scatterClass(
        name=label.strip(),
        mode='markers',
        marker=dict(
            size=5,
            line=dict(
                width=0.5
            ),
            opacity=1
        ),
        text=ids,
        **coords
    )
    traces.append(trace)
layout = go.Layout(
    hovermode='closest',
    title=tensor_name,
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0
    )
)
fig = go.Figure(data=traces, layout=layout)
plot = py.iplot(fig, filename=tensor_name)
print('{}: {}'.format(tensor_name, plot.resource))

9fc0cd0706914a62b41f67b9360d8833: https://plot.ly/~6X5uQKc1/28


In [3]:
import colorlover as cl

In [21]:
list(cl.scales['11']['div']['Spectral'])

['rgb(158,1,66)',
 'rgb(213,62,79)',
 'rgb(244,109,67)',
 'rgb(253,174,97)',
 'rgb(254,224,139)',
 'rgb(255,255,191)',
 'rgb(230,245,152)',
 'rgb(171,221,164)',
 'rgb(102,194,165)',
 'rgb(50,136,189)',
 'rgb(94,79,162)']